In [1]:
# Load Data
import pandas as pd
import ray

ray.shutdown()
ray.init(object_store_memory=6000000000) # set object store memory to 4GB

data = pd.read_pickle('C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\data\\pickle_files\\full_dataset.pkl')
sample = data.sample(10001, random_state=26)

# tansform 'Rating' from Categorical to float
import numpy as np
sample["Rating"] = pd.to_numeric(sample["Rating"], errors='coerce')

# drop all rows with inf/-inf values!
import numpy as np
sample = sample[(sample != np.inf).all(axis=1)]
sample = sample[(sample != -np.inf).all(axis=1)]

#get rid of whitespace to draw tree later
Eq_Stylebox = sample['Eq_Stylebox_Long'].astype('object').replace(' ','_', regex=True)
sample['Eq_Stylebox_Long'] = Eq_Stylebox.astype('category')

# shifting target variable to predict next month
sample['returns'] = sample.returns.shift(-1)
sample = sample.drop(sample.tail(1).index)


X = sample.drop('returns', axis=1)
y = sample['returns']

# create dummies in case of categorical data
dummy_needed = [#'Rating',
                'Financial_Health_Grade_Long',
                 'Growth_Grade_Long',
                 'Profitability_Grade_Long',
                 'Eq_Stylebox_Long']

X = pd.get_dummies(X, columns=dummy_needed)

# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 26)
X_train = ray.put(X_train)
y_train = ray.put(y_train)

2023-04-13 16:49:17,016	INFO worker.py:1553 -- Started a local Ray instance.


In [2]:
%%time
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from sklearn.model_selection import cross_val_score, KFold
import numpy as np


# Define the search space for hyperparameters
search_space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.0001), np.log(0.9)),
    "max_depth": hp.choice("max_depth", range(1, 16)),
    "n_estimators": hp.choice("n_estimators", range(100, 10000)),
    "subsample": hp.uniform("subsample", 0.5, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "colsample_bylevel": hp.uniform("colsample_bylevel", 0.5, 1),
    "reg_alpha": hp.loguniform("reg_alpha", np.log(0.001), np.log(100)),
    "reg_lambda": hp.loguniform("reg_lambda", np.log(0.001), np.log(100)),
}


kfolds = KFold(n_splits=5, shuffle=True, random_state=26)

# Define the objective function to optimize
def objective(config):
    model = xgb.XGBRFRegressor(**config, n_jobs=-1)
    score = cross_val_score(model, X=ray.get(X_train), y=ray.get(y_train), scoring="neg_mean_squared_error", cv=kfolds)
    rmse = np.sqrt(-np.mean(score))
    tune.report(rmse=rmse)

# Define the search algorithm
search_alg = HyperOptSearch(space=search_space, metric="rmse", mode="min")
# to limit number of cores, uncomment and set max_concurrent 
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

# Define the hyperparameter tuning trials object
trials = Trials()

# Define the configuration for Ray Tune
config = {
    'num_samples': 10,
    'config': search_space,
    'search_alg': search_alg,
    'scheduler': scheduler,
    'resources_per_trial': {'cpu': 4},
    'metric': 'rmse',
    'mode': 'min',
    'verbose': 1,
    'name': 'xgboost_tuning_RF',
    'stop': {'training_iteration': 10},
    'local_dir': './ray_results_RF',
}

# Start the hyperparameter tuning using Ray Tune
analysis = tune.run(objective, **config)

# Print the best hyperparameters found during the search
best_params = analysis.get_best_config(metric='rmse')
print("Best hyperparameters found:")
print(best_params)

2023-04-13 18:04:50,853	WARNING trial_runner.py:964 -- Trial Runner checkpointing failed: [WinError 5] Zugriff verweigert: 'C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\ml_prediction\\ray_results_RF\\xgboost_tuning_RF\\.tmp_searcher_ckpt' -> 'C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\ml_prediction\\ray_results_RF\\xgboost_tuning_RF\\searcher-state-2023-04-13_16-49-39.pkl'
2023-04-13 18:11:55,357	WARNING trial_runner.py:964 -- Trial Runner checkpointing failed: [WinError 5] Zugriff verweigert: 'C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\ml_prediction\\ray_results_RF\\xgboost_tuning_RF\\.tmp_checkpoint' -> 'C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\ml_prediction\\ray_results_RF\\xgboost_tuning_RF\\experiment_state-2023-04-13_16-49-39.json'
2023-04-13 18:12:00,422	WARNING trial_runner.py:964 -- Trial Runner checkpointing failed: [WinError 5] Zugriff verweigert: 'C:\\Users\\manue\\switchdrive\\Mutual Funds Project\\ml_prediction\\ray_results_RF\\xgbo

Best hyperparameters found:
{'learning_rate': 0.6390238177217922, 'max_depth': 15, 'n_estimators': 977, 'subsample': 0.5780787559448868, 'colsample_bytree': 0.8465078201572098, 'colsample_bylevel': 0.9229226398911605, 'reg_alpha': 0.0015804680567206015, 'reg_lambda': 0.4584670535706959}
Wall time: 1h 42min 19s


In [3]:
best_params = analysis.get_best_config(metric='rmse')
print("Best hyperparameters found:")
print(best_params)

Best hyperparameters found:
{'learning_rate': 0.6390238177217922, 'max_depth': 15, 'n_estimators': 977, 'subsample': 0.5780787559448868, 'colsample_bytree': 0.8465078201572098, 'colsample_bylevel': 0.9229226398911605, 'reg_alpha': 0.0015804680567206015, 'reg_lambda': 0.4584670535706959}
